In [ ]:
# Import des librairies principales
import pandas as pd
import numpy as np

# Séparation train/test
from sklearn.model_selection import train_test_split

# Métriques d'évaluation
from sklearn.metrics import (
    accuracy_score, recall_score, f1_score,
    precision_score, balanced_accuracy_score,
    matthews_corrcoef, classification_report
)

# Modèle XGBoost
from xgboost import XGBClassifier
import xgboost as xgb





In [ ]:
# Chargement du fichier CSV
df = pd.read_csv("transactions.csv")


In [ ]:
df.head()


In [ ]:
# Convertir la colonne en datetime avec gestion du fuseau horaire
# Pandas détecte le 'Z' et met la colonne en UTC
df["transaction_time"] = pd.to_datetime(df["transaction_time"], utc=True) #transformation de la colonne transaction_time en datetime pour avoir le moment exact de chaque transaction #


In [ ]:
import pandas as pd

# Mapping des fuseaux horaires IANA pour les pays du dataset
TZ_MAP = {
    'FR': 'Europe/Paris',
    'US': 'America/New_York',
    'TR': 'Europe/Istanbul',
    'PL': 'Europe/Warsaw',
    'ES': 'Europe/Madrid',
    'IT': 'Europe/Rome',
    'RO': 'Europe/Bucharest',
    'GB': 'Europe/London',
    'NL': 'Europe/Amsterdam',
    'DE': 'Europe/Berlin'
}

# 1. Charger le dataset (Assurez-vous que cette étape est effectuée en amont)
# df = pd.read_csv("transactions.csv")

# 2. Préparation : Convertir l'heure en datetime et s'assurer qu'elle est localisée en UTC.
df["transaction_time"] = pd.to_datetime(df["transaction_time"], utc=True)

# 3. Fonction d'extraction et de conversion
def get_local_feature(row, feature_type):
    """Convertit l'horodatage UTC en local et extrait une feature spécifique."""
    country_code = row['country']
    tz_name = TZ_MAP.get(country_code, 'UTC') # Sécurité : utilise UTC si le pays n'est pas mappé
    
    # Convertir l'horodatage UTC au fuseau horaire local
    local_dt = row['transaction_time'].tz_convert(tz_name)
    
    # Extraire la feature
    if feature_type == 'hour':
        return local_dt.hour
    elif feature_type == 'dayofweek':
        return local_dt.dayofweek
    elif feature_type == 'day_name':
        return local_dt.day_name()
    elif feature_type == 'month':
        return local_dt.month
    elif feature_type == 'year':
        return local_dt.year
    elif feature_type == 'day':
        return local_dt.day
    # Retourner l'objet datetime local complet (utile pour la vérification)
    elif feature_type == 'full_local':
        return local_dt

# 4. Extraction des features locales dans le DataFrame :
df["hour_local"] = df.apply(lambda row: get_local_feature(row, 'hour'), axis=1)
df["dayofweek_local"] = df.apply(lambda row: get_local_feature(row, 'dayofweek'), axis=1)
# df["month_local"] = df.apply(lambda row: get_local_feature(row, 'month'), axis=1)
# df["year_local"] = df.apply(lambda row: get_local_feature(row, 'year'), axis=1)
df["day_local"] = df.apply(lambda row: get_local_feature(row, 'day'), axis=1)


In [ ]:
print(df.columns)


In [ ]:
df = df.sort_values(["user_id","transaction_time"]).reset_index(drop=True) #Classe


In [ ]:

# df["hour"] = df["transaction_time"].dt.hour #on extrait l'heure de la transaction pour repérer les périodes ou la fraude est plus fréquente #OK

# df["hour"] = df["local_tx_date"].dt.hour #on extrait l'heure de la transaction pour repérer les périodes ou la fraude est plus fréquente #OK

# df["dayofweek"] = df["transaction_time"].dt.dayofweek #on extrait le jour de la semaine pour voir si certaine journée (comme le week end) on plus de fraude #OK

# df["dayofweek"] = df["local_tx_date"].dt.dayofweek #on extrait le jour de la semaine pour voir si certaine journée (comme le week end) on plus de fraude #OK

df["is_night"] = ((df["hour_local"] >= 22) | (df["hour_local"] <= 5)) * 1 #ici créatiion d'une variable qui vaudra 1 si la transaction a lieu la nuit (de 22h a 5h du matin) le test logique  donnera "True" si entre 0 et 5 sinon False et ensuite transformation du resultat en 1/0  utilisable par un modele #OK

df["avg_amount_user_past"] = (df.groupby("user_id")["amount"].expanding().mean().shift(1).reset_index(level=0, drop=True)) # calcule la somme cumulée et le nombre de transactions passées, on les divise, et on obtient la moyenne passée sans aucune donnée future.

df["amount_diff_user_avg"] = df["amount"] - df["avg_amount_user_past"]# mesure si le montant actuel est différent du montant moyen habituel de l'utilisateur # OK

df["is_new_account"] = (df["account_age_days"] < 30) * 1 #verifie si c'est un nouveau compte ou pas , test logique pour voir si il a moin de 30 jour et transformation du resultat en 1/0 #OK

df["security_mismatch_score"] = (df["avs_match"] == 0) * 1 + (df["cvv_result"] == 0) * 1 # ici ca calcule un score de risque en comptant combien de vérification on échoué (avs ou cvv) un total de 0,1,2 #OK
#________________________________________________________________________________________________________________________________________
df["user_fraud_count"] = (df.groupby("user_id")["is_fraud"].cumsum().shift(1).fillna(0)) #calcul combien de fraudes un utilisateur a deja fait au total #ok

df["user_has_fraud_history"] = (df["user_fraud_count"] > 0) * 1 # montre si l'utilisateur a deja fraudé au moin 1 fios 0/1

df["user_tx_count"] = df.groupby("user_id").cumcount() #combien de transaction l'utilisateur avait avant celle ci (compteur historique)

df["user_fraud_rate"] = (df["user_fraud_count"] / df["user_tx_count"]).fillna(0) #nombre de fraude déjà commises avant celle ci
df["user_fraud_rate"] = df["user_fraud_rate"].replace([np.inf, -np.inf], 0)


#_________________________________________________________________________________________________________________________________________

df["country_bin_mismatch"] = (df["country"] != df["bin_country"]) * 1 #verifie que le pays de la carte (le bin) ne correspond pas au pays de la transaction 0/1

df["distance_amount_ratio"] = df["shipping_distance_km"] / (df["amount"] + 1) #mesure si la distance d'expedition est plus importante que le montant de la transaction

df["amount_delta_prev"] = df.groupby("user_id")["amount"].diff().fillna(0) #mesure la différence entre le montant actuel et le montant de la transaction précédente du meme utilisateur pour reperer les changement de comportement
#__________________________________________________________________________________________________________________________________________
df["channel_changed"] = (df["channel"] != df.groupby("user_id")["channel"].shift()).astype(int)# vaux 1 si le canal de transaction change #BINAIRE NECESSAIRE

#__________________________________________________________________________________________________________________________________________
df["time_since_last"] = df.groupby("user_id")["transaction_time"].diff().dt.total_seconds() #mesure le nombre de secondes entre la transaction actuelle et la derniere

df["transaction_count_cum"] = df.groupby("user_id").cumcount() + 1
#________________________________________________________________________________________________________________________________



In [ ]:

# Tri (Toujours nécessaire)
df = df.sort_values(["user_id", "transaction_time"])

#  Fonction corrigée sans shift, avec closed='left'
def get_rolling_count_safe(g, window):
    # closed='left' signifie : regarde du passé jusqu'à maintenant,
    # mais EXCLUT la transaction actuelle du compte.
    return pd.Series(
        g.set_index("transaction_time")["amount"]
         .rolling(window, closed='left')
         .count()
         .values,
        index=g.index
    )

#  Application
# On sélectionne les colonnes avant le apply pour éviter les warnings/erreurs
cols_needed = ["transaction_time", "amount"]

df["tx_last_24h"] = df.groupby("user_id", group_keys=False)[cols_needed].apply(
    lambda g: get_rolling_count_safe(g, "24h")
)

df["tx_last_7d"] = df.groupby("user_id", group_keys=False)[cols_needed].apply(
    lambda g: get_rolling_count_safe(g, "7d")
)

df["tx_last_30d"] = df.groupby("user_id", group_keys=False)[cols_needed].apply(
    lambda g: get_rolling_count_safe(g, "30d")
)

# Remplacer les NaN (premières lignes) par 0
df[["tx_last_24h", "tx_last_7d", "tx_last_30d"]] = df[["tx_last_24h", "tx_last_7d", "tx_last_30d"]].fillna(0)


In [ ]:
# RÉPARATION : On remet l'index à zéro pour récupérer 'transaction_time' si elle était cachée
df = df.reset_index()

# Si 'index' a été créé en trop lors du reset, on le supprime (optionnel mais propre)
if "index" in df.columns:
    df = df.drop(columns=["index"])

#  S'assurer que c'est bien une date (sinon le rolling plante)
df["transaction_time"] = pd.to_datetime(df["transaction_time"])

#  Tri (Obligatoire)
df = df.sort_values(["user_id", "transaction_time"])

#  Calcul Optimisé (Boucle propre)
# On utilise closed='left' pour exclure la transaction actuelle du compte (remplace le shift)
windows = {"24h": "tx_last_24h", "7d": "tx_last_7d", "30d": "tx_last_30d"}
cols_needed = ["transaction_time", "amount"]

for window, col_name in windows.items():
    df[col_name] = df.groupby("user_id", group_keys=False)[cols_needed].apply(
        lambda g: pd.Series(
            g.set_index("transaction_time")["amount"]
             .rolling(window, closed='left')
             .count()
             .values,
            index=g.index
        )
    ).fillna(0)

# Vérification finale
print(df[["user_id", "transaction_time", "tx_last_24h", "tx_last_7d", "tx_last_30d"]].head())


In [ ]:
df.head(2)



In [ ]:
df = df.drop(columns=["transaction_time"]) #Suppression de la colonne transaction time
df.drop(columns=["transaction_id"], inplace=True)
df = pd.get_dummies(
    df, columns=["country", "bin_country", "channel", "merchant_category",],
    drop_first=False
)



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, balanced_accuracy_score, matthews_corrcoef, confusion_matrix, classification_report
# Target
y = df["is_fraud"]

# Features (tout sauf la target)
X = df.drop(columns=["is_fraud"])




In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
# Calcul poids de classe
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

model = xgb.XGBClassifier(
    n_estimators=400,
    learning_rate=0.05,
    max_depth=4,
    min_child_weight=12,   # limite les feuilles trop petites → moins d’overfit
    gamma=3,               # empêche les splits inutiles → régularisation
    subsample=0.7,         # chaque arbre voit moins d’échantillons → généralisation
    colsample_bytree=0.7,  # chaque arbre voit moins de features → moins d’overfit
    objective="binary:logistic",
    eval_metric="logloss",
    scale_pos_weight=12,   # moins agressif que 15 → meilleur équilibre
    random_state=42,
    n_jobs=-1
)





In [ ]:
df["time_since_last"] = df["time_since_last"].fillna(0)



In [ ]:
# 1. Train
model.fit(X_train, y_train)

# 2. Predict
y_train_pred = model.predict(X_train)
y_test_pred  = model.predict(X_test)

# 3. Metrics
print("===== TRAIN RESULTS =====")
print("Accuracy :", accuracy_score(y_train, y_train_pred))
print("Recall   :", recall_score(y_train, y_train_pred))
print("Precision:", precision_score(y_train, y_train_pred))
print("F1-score :", f1_score(y_train, y_train_pred))

print("\n===== TEST RESULTS =====")
print("Accuracy :", accuracy_score(y_test, y_test_pred))
print("Recall   :", recall_score(y_test, y_test_pred))
print("Precision:", precision_score(y_test, y_test_pred))
print("F1-score :", f1_score(y_test, y_test_pred))



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Récupération des importances XGBoost
importance = model.get_booster().get_score(importance_type='gain')

#  DataFrame trié
importance_df = pd.DataFrame(
    importance.items(),
    columns=["feature", "importance"]
).sort_values("importance", ascending=False)

#  Affichage du tableau complet
display(importance_df)

#  Plot propre sans warning
plt.figure(figsize=(10, len(importance_df) * 0.3))  # hauteur auto-adaptée
sns.barplot(
    data=importance_df,
    x="importance",
    y="feature",
    hue="feature",        # pour éviter le warning
    dodge=False,
    legend=False,
    palette="viridis"
)
plt.title("Feature Importance (gain) – All Features")
plt.xlabel("Importance (gain)")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()



In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Conversion des booléens en int (sinon heatmap plante)
df_corr = df.copy()
bool_cols = df_corr.select_dtypes(include=['bool']).columns
df_corr[bool_cols] = df_corr[bool_cols].astype(int)

# 2. Calcul des corrélations
corr_matrix = df_corr.corr()

# 3. Heatmap
plt.figure(figsize=(22, 18))
sns.heatmap(
    corr_matrix,
    cmap="coolwarm",
    annot=False,
    vmin=-1, vmax=1,
    linewidths=0.1
)

plt.title("Heatmap de Corrélation – Dataset Fraude", fontsize=16)
plt.tight_layout()
plt.show()



In [ ]:
# 1. Obtenir les corrélations avec la variable cible
target_corr = corr_matrix['is_fraud'].drop('is_fraud')

# 2. Trier par valeur absolue et ne garder que les 20 premières
top_features = target_corr.abs().sort_values(ascending=False).head(20).index
target_corr_top = target_corr.loc[top_features]

# 3. Visualisation (Bar Plot au lieu de Heatmap)
plt.figure(figsize=(10, 8))
sns.barplot(x=target_corr_top.values, y=target_corr_top.index, palette="coolwarm")
plt.title("Top 20 Corrélations avec 'is_fraud'")
plt.show()


 Typologies de fraude : antécédents, géographiques, montants, autres facteurs (canal web, promo)

In [ ]:
# 1. Utiliser clustermap au lieu de heatmap
plt.figure(figsize=(22, 18))
sns.clustermap(
    corr_matrix,
    cmap="coolwarm",
    annot=False,
    vmin=-1, vmax=1,
    linewidths=0.1,
    figsize=(18, 18) # Utiliser figsize dans clustermap lui-même
)
plt.suptitle("Clustermap de Corrélation – Regroupement des Caractéristiques", y=1.02, fontsize=16)
plt.show()


In [ ]:
import optuna
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, recall_score
from sklearn.model_selection import train_test_split

# Fonction objectif Optuna
def objective(trial):

    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1500),
        "max_depth": trial.suggest_int("max_depth", 2, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "gamma": trial.suggest_float("gamma", 0, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "scale_pos_weight": trial.suggest_float("scale_pos_weight", 5, 50),

        "objective": "binary:logistic",
        "eval_metric": "logloss",
        "random_state": 42,
        "n_jobs": -1
    }

    model = XGBClassifier(**params)

    # Train
    model.fit(X_train, y_train)

    # Predictions
    preds = model.predict(X_test)

    # Score orienté fraude : Recall + F1
    recall = recall_score(y_test, preds)
    f1 = f1_score(y_test, preds)

    # On maximise : trade-off F1 + Recall
    return (0.6 * recall) + (0.4 * f1)



In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=150)



In [ ]:
print("Best Score:", study.best_value)
print("Best Params:", study.best_params)



In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import (
    recall_score,
    precision_score,
    f1_score,
    classification_report,
    confusion_matrix
)

# ===============================
# 1. Modèle XGBoost – Best Optuna Params
# ===============================

model = XGBClassifier(
    n_estimators=360,
    max_depth=3,
    learning_rate=0.08988925976027803,
    gamma=6.832453848633769,
    min_child_weight=16,
    subsample=0.9989920858950934,
    colsample_bytree=0.5418940975093512,
    scale_pos_weight=5.003569800495007,
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)

# ===============================
# 2. Entraînement
# ===============================
model.fit(X_train, y_train)

# ===============================
# 3. Prédictions (seuil par défaut 0.5)
# ===============================
y_train_pred = model.predict(X_train)
y_test_pred  = model.predict(X_test)

# ===============================
# 4. Scores
# ===============================
print("===== TRAIN RESULTS =====")
print("Recall   :", recall_score(y_train, y_train_pred))
print("Precision:", precision_score(y_train, y_train_pred))
print("F1-score :", f1_score(y_train, y_train_pred))

print("\n===== TEST RESULTS =====")
print("Recall   :", recall_score(y_test, y_test_pred))
print("Precision:", precision_score(y_test, y_test_pred))
print("F1-score :", f1_score(y_test, y_test_pred))

# ===============================
# 5. Confusion Matrix
# ===============================
print("\n===== CONFUSION MATRIX =====")
print(confusion_matrix(y_test, y_test_pred))

print("\n===== CLASSIFICATION REPORT =====")
print(classification_report(y_test, y_test_pred))




In [ ]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import recall_score, precision_score, f1_score
import pandas as pd


models = {
    "XGBoost": XGBClassifier(
        n_estimators=360,
        max_depth=3,
        learning_rate=0.08988925976027803,
        gamma=6.832453848633769,
        min_child_weight=16,
        subsample=0.9989920858950934,
        colsample_bytree=0.5418940975093512,
        scale_pos_weight=5.003569800495007,
        objective="binary:logistic",
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1
    ),

    "RandomForest": RandomForestClassifier(
        n_estimators=300,
        max_depth=12,
        class_weight="balanced",
        random_state=42,
        n_jobs=-1
    ),

    "ExtraTrees": ExtraTreesClassifier(
        n_estimators=300,
        max_depth=None,
        class_weight="balanced",
        random_state=42,
        n_jobs=-1
    ),

    "HistGradientBoosting": HistGradientBoostingClassifier(
        max_depth=10,
        learning_rate=0.1,
        l2_regularization=1.0,
        random_state=42
    ),

    "CatBoost": CatBoostClassifier(
        iterations=400,
        depth=6,
        learning_rate=0.05,
        loss_function="Logloss",
        verbose=0,
        random_state=42
    )
}

results = []

def evaluate_model(name, model):
    preds = model.predict(X_test)
    recall = recall_score(y_test, preds)
    precision = precision_score(y_test, preds)
    f1 = f1_score(y_test, preds)
    results.append([name, recall, precision, f1])


print("=== TRAINING MODELS ===")
for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    evaluate_model(name, model)

results_df = pd.DataFrame(results, columns=["Model", "Recall", "Precision", "F1"])

print("\n=== RESULTS COMPARISON ===")
print(results_df.sort_values("F1", ascending=False))



In [ ]:
# ============================================================
# SMOTETomek robuste
# Gère automatiquement les datasets :
# - full numériques
# - mixtes numériques / catégorielles
# ============================================================

import numpy as np
from sklearn.impute import SimpleImputer
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTENC, SMOTE

# ============================================================
# 1. Copies de sécurité
# ============================================================

X_train_res = X_train.copy()
y_train_res = y_train.copy()

# ============================================================
# 2. Détection des types de colonnes
# ============================================================

cat_cols = X_train_res.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = X_train_res.columns.difference(cat_cols).tolist()

print("Colonnes catégorielles :", cat_cols)
print("Colonnes numériques :", num_cols)

# ============================================================
# 3. Imputation des valeurs manquantes
# ============================================================

# Numériques -> médiane
num_imputer = SimpleImputer(strategy="median")
X_train_res[num_cols] = num_imputer.fit_transform(X_train_res[num_cols])

# Catégorielles -> valeur la plus fréquente (si présentes)
if len(cat_cols) > 0:
    cat_imputer = SimpleImputer(strategy="most_frequent")
    X_train_res[cat_cols] = cat_imputer.fit_transform(X_train_res[cat_cols])

print("Nombre total de NaN restants :", X_train_res.isna().sum().sum())

# ============================================================
# 4. Sélection automatique de la stratégie SMOTE
# ============================================================

if len(cat_cols) > 0:
    # Cas colonnes catégorielles non encodées
    cat_indices = [X_train_res.columns.get_loc(col) for col in cat_cols]
    print("Stratégie : SMOTENC + TomekLinks")

    smote = SMOTENC(
        categorical_features=cat_indices,
        random_state=42,
        k_neighbors=5
    )
else:
    # Cas full numérique
    print("Stratégie : SMOTE classique + TomekLinks")

    smote = SMOTE(
        random_state=42,
        k_neighbors=5
    )

smote_tomek = SMOTETomek(
    smote=smote,
    random_state=42,
    n_jobs=-1
)

# ============================================================
# 5. Resampling (TRAIN uniquement)
# ============================================================

X_train_balanced, y_train_balanced = smote_tomek.fit_resample(
    X_train_res,
    y_train_res
)

# ============================================================
# 6. Résumé final
# ============================================================

print("\n================ RÉSULTAT =================")
print(
    f"Avant  : {X_train_res.shape[0]:,} lignes | "
    f"{y_train_res.sum():,} fraudes ({y_train_res.mean():.2%})"
)

print(
    f"Après  : {X_train_balanced.shape[0]:,} lignes | "
    f"{y_train_balanced.sum():,} fraudes ({y_train_balanced.mean():.2%})"
)



In [ ]:
# ============================================================
# XGBOOST FINAL — FRAUDE DETECTION (SETUP VALIDÉ)
# ============================================================

import numpy as np
import xgboost as xgb

from sklearn.metrics import (
    roc_auc_score,
    classification_report,
    confusion_matrix
)

# ============================================================
# 1. SCALE_POS_WEIGHT (SUR TRAIN BALANCÉ)
# ============================================================

neg, pos = np.bincount(y_train_balanced)
scale_pos_weight = neg / pos

print("scale_pos_weight :", scale_pos_weight)

# ============================================================
# 2. MODÈLE XGBOOST — PARAMÈTRES OPTUNA
# ============================================================

xgb_model = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="aucpr",                 # adapté dataset déséquilibré
    n_estimators=360,
    max_depth=3,
    learning_rate=0.08988925976027803,
    gamma=6.832453848633769,
    min_child_weight=16,
    subsample=0.9989920858950934,
    colsample_bytree=0.5418940975093512,
    scale_pos_weight=scale_pos_weight,   # cohérent avec train rééquilibré
    tree_method="hist",
    random_state=42,
    n_jobs=-1
)

# ============================================================
# 3. ENTRAÎNEMENT (TRAIN SMOTÉ UNIQUEMENT)
# ============================================================

print("\nEntraînement XGBoost...")
xgb_model.fit(X_train_balanced, y_train_balanced)

# ============================================================
# 4. PRÉDICTIONS SUR LE TEST RÉEL (NON SMOTÉ)
# ============================================================

y_proba = xgb_model.predict_proba(X_test)[:, 1]

# ============================================================
# 5. AJUSTEMENT DU SEUIL (ORIENTÉ RECALL)
# ============================================================

threshold = 0.30
y_pred = (y_proba >= threshold).astype(int)

print("Seuil utilisé :", threshold)

# ============================================================
# 6. ÉVALUATION FINALE
# ============================================================

print("\n================ PERFORMANCE TEST =================")

print("ROC AUC :", roc_auc_score(y_test, y_proba))

print("\nConfusion Matrix :")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report :")
print(classification_report(y_test, y_pred, digits=4))





In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Matrice de confusion brute
cm = confusion_matrix(y_test, y_pred)

# Normalisation par ligne (vraie classe)
cm_percent = cm.astype(float) / cm.sum(axis=1, keepdims=True) * 100

# Affichage
plt.figure(figsize=(5, 4))
plt.imshow(cm_percent)
plt.colorbar(format="%.0f%%")

plt.xticks([0, 1], ["Non fraude", "Fraude"])
plt.yticks([0, 1], ["Non fraude", "Fraude"])

plt.xlabel("Prédiction")
plt.ylabel("Vraie classe")
plt.title("Matrice de confusion (%) – XGBoost (seuil ajusté)")

# Valeurs en %
for i in range(2):
    for j in range(2):
        plt.text(
            j,
            i,
            f"{cm_percent[i, j]:.2f}%",
            ha="center",
            va="center",
            fontsize=11
        )

plt.tight_layout()
plt.show()




 La matrice de confusion normalisée montre que le modèle laisse passer plus de 99,6 % des transactions légitimes tout en détectant plus de 81 % des fraudes, ce qui constitue un compromis pertinent entre risque financier et impact client.

In [ ]:
# Récupération des importances (gain)
importances = xgb_model.get_booster().get_score(importance_type="gain")

# DataFrame
fi_df = pd.DataFrame({
    "Feature": importances.keys(),
    "Gain": importances.values()
})

# Normalisation en %
fi_df["Importance_%"] = fi_df["Gain"] / fi_df["Gain"].sum() * 100

# Sélection du Top 15
fi_top15 = fi_df.sort_values("Importance_%", ascending=False).head(15)

# Tri pour affichage horizontal
fi_top15 = fi_top15.sort_values("Importance_%", ascending=True)

# Plot
plt.figure(figsize=(8, 6))
plt.barh(fi_top15["Feature"], fi_top15["Importance_%"])
plt.xlabel("Importance (%)")
plt.ylabel("Feature")
plt.title("Feature Importance – XGBoost (Top 15, gain en %)")
plt.tight_layout()
plt.show()




 L’analyse d’importance montre que le modèle se base principalement sur des signaux de sécurité du paiement et de cohérence transactionnelle, complétés par l’historique utilisateur et le contexte, ce qui est conforme aux pratiques antifraude

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score
import numpy as np

# ROC + AUC
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
auc_score = roc_auc_score(y_test, y_proba)

# Trouver le point correspondant au seuil choisi (ex: 0.30)
threshold_target = 0.30
idx = np.argmin(np.abs(thresholds - threshold_target))

# Plot
plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr, label=f"XGBoost ROC (AUC = {auc_score:.4f})")
plt.plot([0, 1], [0, 1], linestyle="--", label="Random")

# Annotation du seuil
plt.scatter(fpr[idx], tpr[idx])
plt.text(
    fpr[idx], tpr[idx],
    f"  seuil={threshold_target}",
    verticalalignment="bottom"
)

plt.xlabel("False Positive Rate (FPR)")
plt.ylabel("True Positive Rate (TPR)")
plt.title("ROC Curve – XGBoost (test non smoté)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()




 La courbe ROC montre que le modèle discrimine très bien les classes avec une AUC proche de 0,98. Le seuil opérationnel n’est pas choisi sur la ROC mais en fonction du compromis métier, ici orienté recall, ce qui est matérialisé par le point à 0,3.

In [ ]:
import json
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def analyze_random_transaction(model, X_data, y_data, threshold=0.3):
    """
    Sélectionne une transaction au hasard, prédit la fraude
    et explique la décision via SHAP, avec comparaison vérité terrain.
    """

    # ============================================================
    # 1. Sélection aléatoire d'une observation
    # ============================================================

    random_idx = np.random.choice(X_data.index)
    row_data = X_data.loc[[random_idx]]  # format DataFrame
    true_label = y_data.loc[random_idx]

    # Conversion en JSON lisible
    row_dict = row_data.iloc[0].astype(object).to_dict()
    for k, v in row_dict.items():
        if isinstance(v, (np.integer, np.floating)):
            row_dict[k] = float(v)

    json_output = json.dumps(row_dict, indent=4)

    print(f"--- OBSERVATION (INDEX: {random_idx}) ---")
    print(f"Vraie classe : {'FRAUDE' if true_label == 1 else 'NON FRAUDE'}")
    print("Données (JSON) :")
    print(json_output)
    print("-" * 60)

    # ============================================================
    # 2. Prédiction du modèle
    # ============================================================

    prob_fraud = model.predict_proba(row_data)[:, 1][0]
    pred_label = 1 if prob_fraud >= threshold else 0

    print(f"\n--- PRÉDICTION (Seuil = {threshold}) ---")
    print(f"Probabilité de fraude : {prob_fraud:.4f}")
    print(f"Classe prédite        : {'FRAUDE' if pred_label == 1 else 'NON FRAUDE'}")
    print("-" * 60)

    # ============================================================
    # 3. COMPARAISON PRÉDICTION vs RÉALITÉ (IMPORTANT)
    # ============================================================

    print("\n--- VERDICT ---")

    if pred_label == true_label:
        if pred_label == 1:
            verdict = "VRAI POSITIF (fraude correctement détectée)"
        else:
            verdict = "VRAI NÉGATIF (transaction légitime correctement classée)"
    else:
        if pred_label == 1:
            verdict = "FAUX POSITIF (transaction légitime bloquée à tort)"
        else:
            verdict = "FAUX NÉGATIF (fraude non détectée)"

    print(f"Résultat : {verdict}")
    print("-" * 60)

    # ============================================================
    # 4. EXPLICATION SHAP (locale)
    # ============================================================

    print("\n--- EXPLICATION DE LA DÉCISION (SHAP) ---")

    explainer = shap.TreeExplainer(model)
    shap_values = explainer(row_data)

    contributions = pd.DataFrame({
        "Feature": X_data.columns,
        "Valeur_Feature": row_data.iloc[0].values,
        "Impact_SHAP": shap_values.values[0]
    })

    contributions["Abs_Impact"] = contributions["Impact_SHAP"].abs()

    top_contributions = (
        contributions
        .sort_values(by="Abs_Impact", ascending=False)
        .head(10)
        .reset_index(drop=True)
    )

    print("Top 10 des facteurs ayant influencé la décision :")
    display(top_contributions[["Feature", "Valeur_Feature", "Impact_SHAP"]])

    # ============================================================
    # 5. Visualisation SHAP (Waterfall)
    # ============================================================

    plt.figure()
    shap.plots.waterfall(shap_values[0], max_display=10, show=True)


# ============================================================
# EXÉCUTION
# ============================================================

# Assure-toi que xgb_model, X_test et y_test existent
analyze_random_transaction(
    model=xgb_model,
    X_data=X_test,
    y_data=y_test,
    threshold=0.3
)

